In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
from sklearn.metrics import accuracy_score
import pickle

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier

In [4]:
Data = pd.read_csv("Titanic-Dataset.csv")
Data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
Data.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"], inplace=True)

In [6]:
profile = ProfileReport(Data, title="Titanic Report", explorative=True)
profile.to_file("output.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 126.62it/s]


In [7]:
Data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [8]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [9]:
Data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [10]:
Data.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [11]:
Nuemerical_Columns = Data[["Survived", "Pclass", "Age", "SibSp", "Parch", "Fare"]]

In [12]:
Nuemerical_Columns.corr()

,Survived,Pclass,Age,SibSp,Parch,Fare
Survived,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(Data.drop(columns=["Survived"]), Data["Survived"],
                                                   test_size=0.2, random_state=42)

In [14]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [15]:
Y_train.head(10)

331    0
733    0
382    0
704    0
813    0
118    0
536    0
361    0
29     0
55     1
Name: Survived, dtype: int64

In [16]:
Transformer_1 = ColumnTransformer([
    ("Age_Imputer", SimpleImputer(),[2]),
    ("Embarked_Imputer", SimpleImputer(strategy="most_frequent"),[6])],
    remainder="passthrough"
)

In [17]:
Transformer_2 = ColumnTransformer([
    ("Sex_Embarked_Encoded", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), [1,6])
], 
remainder="passthrough")

In [18]:
Transformer_3 = ColumnTransformer([
    ("Scaler", MinMaxScaler(), slice(0,10))
]
)

In [20]:
Model = RandomForestClassifier(n_estimators=100, random_state=42)

In [21]:
PipeLine = Pipeline([
    ("Transformer_1", Transformer_1),
    ("Transformer_2", Transformer_2),
    ("Transformer_3", Transformer_3),
    ("Model", Model)
])

In [22]:
PipeLine.fit(X_train, Y_train)

,steps,"[('Transformer_1', ...), ('Transformer_2', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('Age_Imputer', ...), ('Embarked_Imputer', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [23]:
PipeLine.named_steps

{'Transformer_1': ColumnTransformer(remainder='passthrough',
                   transformers=[('Age_Imputer', SimpleImputer(), [2]),
                                 ('Embarked_Imputer',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'Transformer_2': ColumnTransformer(remainder='passthrough',
                   transformers=[('Sex_Embarked_Encoded',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'Transformer_3': ColumnTransformer(transformers=[('Scaler', MinMaxScaler(), slice(0, 10, None))]),
 'Model': RandomForestClassifier(random_state=42)}

In [24]:
Predicted_Y = PipeLine.predict(X_test)

In [27]:
accuracy_score(Y_test, Predicted_Y)

0.6256983240223464

In [28]:
pickle.dump(PipeLine, open("Titanic.pkl", "wb"))